IMPORTING THE LIBRARIES
torchvision provide sthe necessary datasets for model creations

torch imports the pytorch library. We will import the nn neural network module from torch. optim module provides the built in optimization algorithms like adam will will be used further in the code for model optimization

we import the functional API which provide sthe built in functions like activations functions, relu, sigmoid,pooling an convolutional layer

we import ToTensor function which will convert image data to tensors


In [1]:

import torch,torchvision 
from torch import nn
from torch import optim
import torch.nn.functional as f
from torch.autograd import Variable
from torchvision.transforms import ToTensor



**Converting Image pixels data rows into tensors**

we start by transforming the image data to tensors using torchvision.transforms.compose. we get the MNIST datasets from torchvision.datasets and download the dataset to our computer.

We then split the datset into training and validation sets. we ussually give 80% of the data to the training set.
Using torch.utilis.data.random_split method we divide the whole dataset in the train_data and val_data sets.

We then dowwnload the test dataset

In [2]:
T=torchvision.transforms.Compose(
[torchvision.transforms.ToTensor()]
)
train_data=torchvision.datasets.MNIST('mnist_data',train=True,download=True,transform=T)
val_data=torchvision.datasets.MNIST('mnist_data',train=False,download=True,transform=T)
test_data = torchvision.datasets.MNIST('mnist_data', train=False, download=True, transform=T)
numb_batch = 64
train_dl = torch.utils.data.DataLoader(train_data, batch_size = numb_batch)

test_dl = torch.utils.data.DataLoader(val_data, batch_size = numb_batch, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_data, batch_size = numb_batch)




Dataloaders are iterators that provide the facility to iterate over the whole dataset.
We make three dataloaders, one for test,train and validation each.

In [3]:
from torch.utils.data import DataLoader
loaders = {
'train' : torch.utils.data.DataLoader(train_data,batch_size=100,shuffle=True,num_workers=1),
'val': torch.utils.data.DataLoader(val_data, batch_size=100,shuffle=True, num_workers=1)
}
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x20fe14a7c20>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x20fe14a7a40>}

CREATING THE MODEL

We will make a neural network class that inherits from the nn.Module, the base class for neural networks in pytorch. We define the first convlutional layer with 1 input channel and 32 output channels and krnel size 5.
We define the second convolutional layer with 32 input channels and 32 output channels and kernel size 5
We define the third convolutional layer with 32 input channels and 64 output channels and kernel size 5.
We then declare 2 fully connected layers.The 

In [4]:
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32,64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)#input features in a flattened format 3_3_64 and 256 output features 
        self.fc2 = nn.Linear(256, 10)



    def forward(self, x):
        x = f.relu(self.conv1(x))
        # we apply the RELU Function to the first convolutional layer
        x = f.relu(f.max_pool2d(self.conv2(x), 2))
        #apply the second RELU function on the previous convolutional layer
        x = f.dropout(x, p=0.5, training=self.training)
        #applies dropout regularization to the output of the previous layer
        x = f.relu(f.max_pool2d(self.conv3(x),2))
        #applies the third convolution  to the output of previous layer followed by max pooling then applying the relu functin
        x = f.dropout(x, p=0.5, training=self.training)
        #applies dropout regularization to the output of the previous layer
        x = x.view(-1,3*3*64)
        #reshaping the output
        x = f.relu(self.fc1(x))
        #applying relu again
        x = f.dropout(x, training=self.training)
        x = self.fc2(x)
        return f.log_softmax(x, dim=1)
        #apply the softmax function to the output
        return x
    
net = Net()

criterion = nn.CrossEntropyLoss()
num_classes = 10  # 0-9 digits (assuming MNIST)
learning_rate = 0.01
epochs = 5
batch_size = 64
learning_rate = 0.01
optimizer = optim.Adam(net.parameters(), lr = learning_rate)

TRAINING THE MODEL
We first define a device variiable to check whether the model is trained on the CPU or GPU.
We start training the model and appluing forward propagation and then backward propagation.and finally applying optimizing techniques from the tensor module

In [5]:
# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available
net.to(device)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)

In [6]:
def evaluate_accuracy(data_loader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total


In [7]:
# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available
net.to(device)

total_step = len(train_dl)
print("Training the model...")
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_dl):
        # Move data to the chosen device (CPU or GPU)
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()

        # Print training progress
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')
    val_accuracy = evaluate_accuracy(val_dl)
    print(f'Accuracy of the model on the validation images after epoch {epoch+1}: {val_accuracy:.2f} %')


Training the model...
Epoch [1/5], Step [100/938], Loss: 0.7064
Epoch [1/5], Step [200/938], Loss: 0.7079
Epoch [1/5], Step [300/938], Loss: 0.5939
Epoch [1/5], Step [400/938], Loss: 0.5497
Epoch [1/5], Step [500/938], Loss: 0.4674
Epoch [1/5], Step [600/938], Loss: 0.5960
Epoch [1/5], Step [700/938], Loss: 0.6243
Epoch [1/5], Step [800/938], Loss: 0.3158
Epoch [1/5], Step [900/938], Loss: 0.2322
Accuracy of the model on the validation images after epoch 1: 94.16 %
Epoch [2/5], Step [100/938], Loss: 0.0425
Epoch [2/5], Step [200/938], Loss: 0.0602
Epoch [2/5], Step [300/938], Loss: 0.0659
Epoch [2/5], Step [400/938], Loss: 0.2432
Epoch [2/5], Step [500/938], Loss: 0.0385
Epoch [2/5], Step [600/938], Loss: 0.0608
Epoch [2/5], Step [700/938], Loss: 0.0385
Epoch [2/5], Step [800/938], Loss: 0.0140
Epoch [2/5], Step [900/938], Loss: 0.0540
Accuracy of the model on the validation images after epoch 2: 96.56 %
Epoch [3/5], Step [100/938], Loss: 0.0675
Epoch [3/5], Step [200/938], Loss: 0.039

TESTING DATA ON TRAINING AND VALIDATION DATA

In [8]:

correct = 0
trainaccuracy=evaluate_accuracy(train_dl)
testaccuracy=evaluate_accuracy(test_dl)
valccuracy=evaluate_accuracy(val_dl)
print(f'Accuracy of the model on the training images: {trainaccuracy}')
print(f'Accuracy of the model on the testing images: {testaccuracy}')
print(f'Accuracy of the model on the validation images: {valccuracy}')

  

Accuracy of the model on the training images: 97.93333333333334
Accuracy of the model on the testing images: 97.97
Accuracy of the model on the validation images: 97.97
